# Unidade 2 Trabalho 1 - Avaliando Algoritmos para o Caminho Mais Curto em Grafos Urbanos

<h3> Objetivo Geral </h3>

Avaliar e comparar três algoritmos de menor caminho (OSMnx, Dijkstra tradicional e Dijkstra com min-heap) aplicados a um dos cenários urbanos propostos, com foco em desempenho computacional, similaridade das rotas e impacto ambiental (pegada de carbono).

Os requisitos estão descritos no arquivo: [U2T1.pdf](./U2T1_.pdf)

### DijkstrasAlgorithm

In [2]:
# O(V^2 + E) time | O(V) space - where V is the number of vertices and E is the number of edges in the input graph
def dijkstrasAlgorithm(start, edges):
    """
    Implements Dijkstra's algorithm to find the shortest path from a starting node to all other nodes in a graph.

    Args:
        start (int): The starting node index.
        edges (list of list): Adjacency list representing the graph. Each index corresponds to a vertex,
                              and each entry is a list of pairs [destination, weight].

    Returns:
        list: A list of the shortest distances from the starting node to each node. If a node is not reachable,
              the distance is -1.
    """
    numberOfVertices = len(edges)

    # Initialize the minimum distances for all vertices as infinity
    # except the starting vertex which is set to 0.
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0

    # Keep track of visited nodes to avoid reprocessing them.
    visited = set()

    # Continue processing nodes until all have been visited.
    while len(visited) != numberOfVertices:
        # Find the vertex with the smallest known distance that has not been visited.
        vertex, currentMinDistance = getVertexWithMinDistance(minDistances, visited)

        # If the smallest distance is infinity, all remaining vertices are unreachable.
        if currentMinDistance == float("inf"):
            break

        # Mark the current vertex as visited.
        visited.add(vertex)

        # Iterate through all the neighbors of the current vertex.
        for edge in edges[vertex]:
            destination, distanceToDestination = edge

            # Skip the neighbor if it has already been visited.
            if destination in visited:
                continue

            # Calculate the new potential path distance to the neighbor.
            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            # Update the shortest distance to the neighbor if the new path is shorter.
            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance

    # Replace any remaining infinity distances with -1 to indicate unreachable nodes.
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))


def getVertexWithMinDistance(distances, visited):
    """
    Helper function to find the vertex with the smallest known distance that has not been visited.

    Args:
        distances (list): A list of the shortest known distances to each vertex.
        visited (set): A set of vertices that have already been visited.

    Returns:
        tuple: The index of the vertex with the smallest distance and its distance value.
    """
    currentMinDistance = float("inf")
    vertex = -1

    # Iterate over all vertices to find the one with the smallest distance.
    for vertexIdx, distance in enumerate(distances):
        # Skip the vertex if it has already been visited.
        if vertexIdx in visited:
            continue

        # Update the current minimum distance and vertex if a smaller distance is found.
        if distance <= currentMinDistance:
            vertex = vertexIdx
            currentMinDistance = distance

    return vertex, currentMinDistance

### MinHeap

In [3]:
class MinHeap:
    """
    MinHeap class: Implements a MinHeap data structure to efficiently manage vertices and their distances
    for algorithms like Dijkstra. This implementation keeps track of the position of each vertex using
    a vertex map for constant-time lookups and updates.
    """
    def __init__(self, array):
        """
        Initializes the MinHeap with an input array of (vertex, distance) pairs.

        Args:
            array (list): List of tuples where each tuple is (vertex, distance).
                          The distance is typically initialized to infinity except for the starting vertex.

        Attributes:
            vertexMap (dict): Maps each vertex to its position in the heap for quick access.
            heap (list): List representing the binary heap as an array.
        """
        # Create a vertex map: Maps vertices to their indices in the heap.
        self.vertexMap = {idx: idx for idx in range(len(array))}

        # Build the heap from the input array to satisfy the heap property.
        self.heap = self.buildHeap(array)

    def isEmpty(self):
        """
        Checks if the heap is empty.

        Returns:
            bool: True if the heap is empty, False otherwise.
        """
        return len(self.heap) == 0

    def buildHeap(self, array):
        """
        Builds the heap from an input array in O(n) time.

        Args:
            array (list): List of (vertex, distance) pairs.

        Returns:
            list: The input array transformed into a valid MinHeap.
        """
        # Start from the first parent node and sift down each node.
        firstParentIdx = (len(array) - 2) // 2
        for currentIdx in reversed(range(firstParentIdx + 1)):
            self.siftDown(currentIdx, len(array) - 1, array)
        return array

    def siftDown(self, currentIdx, endIdx, heap):
        """
        Restores the heap property by "sifting down" a node into its correct position.

        Args:
            currentIdx (int): Index of the node to sift down.
            endIdx (int): Last index in the heap.
            heap (list): The heap array.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        childOneIdx = currentIdx * 2 + 1  # Index of the first child
        while childOneIdx <= endIdx:
            # Determine the index of the second child
            childTwoIdx = currentIdx * 2 + 2 if currentIdx * 2 + 2 <= endIdx else -1

            # Choose the smaller child to maintain the min-heap property
            if childTwoIdx != -1 and heap[childTwoIdx][1] < heap[childOneIdx][1]:
                idxToSwap = childTwoIdx
            else:
                idxToSwap = childOneIdx

            # Swap if the child is smaller than the current node
            if heap[idxToSwap][1] < heap[currentIdx][1]:
                self.swap(currentIdx, idxToSwap, heap)
                currentIdx = idxToSwap  # Move to the swapped position
                childOneIdx = currentIdx * 2 + 1  # Update the first child index
            else:
                return

    def siftUp(self, currentIdx, heap):
        """
        Restores the heap property by "sifting up" a node into its correct position.

        Args:
            currentIdx (int): Index of the node to sift up.
            heap (list): The heap array.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        parentIdx = (currentIdx - 1) // 2  # Calculate parent index
        while currentIdx > 0 and heap[currentIdx][1] < heap[parentIdx][1]:
            self.swap(currentIdx, parentIdx, heap)  # Swap with parent
            currentIdx = parentIdx  # Move to the parent's position
            parentIdx = (currentIdx - 1) // 2

    def remove(self):
        """
        Removes and returns the smallest element (root) in the heap.

        Returns:
            tuple: The (vertex, distance) pair with the smallest distance.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        if self.isEmpty():
            return None

        # Swap the root with the last element and remove it
        self.swap(0, len(self.heap) - 1, self.heap)
        vertex, distance = self.heap.pop()
        self.vertexMap.pop(vertex)  # Remove the vertex from the map

        # Restore the heap property
        self.siftDown(0, len(self.heap) - 1, self.heap)
        return vertex, distance

    def swap(self, i, j, heap):
        """
        Swaps two nodes in the heap and updates their positions in the vertexMap.

        Args:
            i (int): Index of the first node.
            j (int): Index of the second node.
            heap (list): The heap array.
        """
        self.vertexMap[heap[i][0]] = j  # Update vertexMap for heap[i]
        self.vertexMap[heap[j][0]] = i  # Update vertexMap for heap[j]
        heap[i], heap[j] = heap[j], heap[i]  # Swap the nodes in the heap

    def update(self, vertex, value):
        """
        Updates the distance of a given vertex and restores the heap property.

        Args:
            vertex (int): The vertex whose distance is to be updated.
            value (int): The new distance value.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        # Update the heap with the new (vertex, value) pair
        self.heap[self.vertexMap[vertex]] = (vertex, value)
        # Restore the heap property by sifting up the updated node
        self.siftUp(self.vertexMap[vertex], self.heap)

### DijkstrasAlgorithmMinHeap

In [4]:
# O((v + e) * log(v)) time | O(v) space — where v is the number
# of vertices and e is the number of edges in the input graph
def dijkstrasAlgorithmMinHeap(start, edges):
    """
    Implements Dijkstra's algorithm to find the shortest paths from a starting vertex to all other vertices
    in a weighted graph. The graph is represented using an adjacency list.

    Args:
        start (int): The starting vertex index.
        edges (list of list): An adjacency list where each index represents a vertex, and each entry
                              is a list of [destination, weight] pairs.

    Returns:
        list: A list of minimum distances from the starting vertex to each vertex in the graph.
              If a vertex is unreachable, its distance is represented as -1.
    """
    # Step 1: Initialize the number of vertices in the graph
    numberOfVertices = len(edges)

    # Step 2: Initialize the minimum distances with infinity
    # Set the starting vertex's distance to 0
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0

    # Step 3: Initialize the MinHeap to track the vertices and their current shortest distances
    minDistancesHeap = MinHeap([(idx, float("inf")) for idx in range(numberOfVertices)])
    minDistancesHeap.update(start, 0)  # Update the starting vertex's distance to 0

    # Step 4: Process vertices until the heap is empty
    while not minDistancesHeap.isEmpty():
        # Extract the vertex with the smallest known distance
        vertex, currentMinDistance = minDistancesHeap.remove()

        # If the current distance is infinity, no further reachable vertices exist
        if currentMinDistance == float("inf"):
            break

        # Step 5: Relaxation - Update distances for all neighboring vertices
        for edge in edges[vertex]:
            destination, distanceToDestination = edge  # Extract neighbor and weight

            # Calculate the new potential path distance
            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            # If the new path is shorter, update the distance and the heap
            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance
                minDistancesHeap.update(destination, newPathDistance)

    # Step 6: Convert unreachable vertices' distances from infinity to -1
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))

### Comparação de Algoritmos de Caminho Mínimo para Serviços de Emergência

In [5]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import networkx as nx
from codecarbon import EmissionsTracker
import os
import pandas as pd

# Caminho do arquivo de emissões
arquivo = "emissions/emissions.csv"

# Verifica se o arquivo de emissões já existe e o apaga para evitar sobrescrita não intencional
if os.path.exists(arquivo):
    os.remove(arquivo)
    print(f"O arquivo '{arquivo}' foi deletado.")

# Cria a pasta 'emissions' se ela ainda não existir
os.makedirs("emissions", exist_ok=True)

# --- Função: conversão de grafo para lista de adjacência indexada ---
def graph_to_indexed_adjacency_list(G, node_to_index, weight_attr='length'):
    """
    Converte um grafo NetworkX para uma lista de adjacência baseada em índices inteiros.
    Cada item da lista contém pares [vizinho, peso].
    """
    adj_list = [[] for _ in range(len(node_to_index))]
    for u, v, data in G.edges(data=True):
        weight = data.get(weight_attr, 1)
        ui = node_to_index[u]
        vi = node_to_index[v]
        adj_list[ui].append([vi, weight])
        if not G.is_directed():
            adj_list[vi].append([ui, weight])
    return adj_list

# --- Preparação do Grafo ---
hospital_coords = (-5.808440, -35.224580)  # Coordenadas do Hospital Walfredo Gurgel (latitude, longitude)

# Baixa o grafo de ruas do entorno do hospital (raio de 6 km), apenas vias acessíveis por carro
G = ox.graph_from_point(hospital_coords, dist=6000, network_type='drive', simplify=True)

# Reprojeta o grafo para um sistema de coordenadas métricas (melhor para cálculos de distância)
G_proj = ox.project_graph(G)

# Cria dicionários de mapeamento entre nós reais e índices inteiros
node_list = list(G_proj.nodes)
node_to_index = {node: idx for idx, node in enumerate(node_list)}
index_to_node = {idx: node for idx, node in enumerate(node_list)}

# --- Lista de Bairros (com endereços) ---
bairros = {
    "Hospital Walfredo Gurgel": "Hospital Walfredo Gurgel, Natal, RN",
    "Felipe Camarão": "Felipe Camarão, Natal, RN",
    "Alecrim": "Alecrim, Natal, RN",
    "Quintas": "Quintas, Natal, RN",
    "Cidade da Esperança": "Cidade da Esperança, Natal, RN",
    "Lagoa Nova": "Lagoa Nova, Natal, RN",
    "Via Costeira": "Via Costeira, Natal, RN",
    "Ponta Negra": "Ponta Negra, Natal, RN",
    "Capim Macio": "Capim Macio, Natal, RN",
    "Neópolis": "Neópolis, Natal, RN",
}

# Geocodifica os endereços dos bairros em coordenadas (lat, lon)
coords = {nome: ox.geocode(endereco) for nome, endereco in bairros.items()}

# Reprojeta os pontos para o mesmo sistema de coordenadas do grafo
projected_points = {}
for nome, (lat, lon) in coords.items():
    point = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326")  # Cria ponto geográfico
    point_proj = point.to_crs(G_proj.graph['crs'])  # Reprojeta para o CRS do grafo
    x, y = point_proj.geometry.iloc[0].x, point_proj.geometry.iloc[0].y
    projected_points[nome] = (x, y)

# Encontra o nó da rede mais próximo de cada bairro
nodes = {
    nome: ox.distance.nearest_nodes(G_proj, x, y)
    for nome, (x, y) in projected_points.items()
}

# Converte o grafo para lista de adjacência indexada
edges = graph_to_indexed_adjacency_list(G_proj, node_to_index)

# Obtém o índice do nó de origem (Hospital Walfredo Gurgel)
start_node = node_to_index[nodes["Hospital Walfredo Gurgel"]]

# --- Execução com rastreamento de emissões ---

# Algoritmo de Dijkstra tradicional (lista de adjacência)
try:
    tracker1 = EmissionsTracker(project_name="Dijkstra Tradicional", output_dir="emissions", save_to_file=True)
    tracker1.start()
    distances_trad = dijkstrasAlgorithm(start_node, edges)
finally:
    tracker1.stop()

# Algoritmo de Dijkstra com Min Heap (mais eficiente)
try:
    tracker2 = EmissionsTracker(project_name="Dijkstra MinHeap", output_dir="emissions", save_to_file=True)
    tracker2.start()
    distances_heap = dijkstrasAlgorithmMinHeap(start_node, edges)
finally:
    tracker2.stop()

# Algoritmo de Dijkstra usando NetworkX (API OSMnx)
try:
    tracker3 = EmissionsTracker(project_name="OSMnx Dijkstra", output_dir="emissions", save_to_file=True)
    tracker3.start()
    distance_osmnx = nx.single_source_dijkstra_path_length(
        G_proj,
        source=index_to_node[start_node],
        weight='length'
    )
finally:
    tracker3.stop()

# Extrai as distâncias do dicionário do NetworkX para cada bairro
dist_osmnx_por_bairro = [
    distance_osmnx.get(index_to_node[node_to_index[nodes[b]]], float("inf"))
    for b in bairros
]


O arquivo 'emissions/emissions.csv' foi deletado.


[codecarbon WARNING @ 15:57:24] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 15:57:24] [setup] RAM Tracking...
[codecarbon INFO @ 15:57:24] [setup] CPU Tracking...
[codecarbon WARNING @ 15:57:25] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 15:57:25] CPU Model on constant consumption mode: AMD Ryzen 5 5600G with Radeon Graphics
[codecarbon WARNING @ 15:57:25] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:57:25] [setup] GPU Tracking...
[codecarbon INFO @ 15:57:25] No GPU found.
[codecarbon INFO @ 15:57:25] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 15:57:25] >>> Tracker's metadata:
[code

### Resultados

In [6]:
df_distancias = pd.DataFrame({
    "bairro": list(bairros.keys()),
    "dijkstra_tradicional_m": [distances_trad[node_to_index[nodes[b]]] for b in bairros],
    "dijkstra_minheap_m":     [distances_heap[node_to_index[nodes[b]]] for b in bairros],
    "dijkstra_osmnx_m":       [dist_osmnx_por_bairro[i] for i, b in enumerate(bairros)],
})

df_distancias.to_csv("emissions/distancias_por_algoritmo.csv", index=False)
df_distancias



,bairro,dijkstra_tradicional_m,dijkstra_minheap_m,dijkstra_osmnx_m
0,Hospital Walfredo Gurgel,0.000000,0.000000,0.000000
1,Felipe Camarão,7168.481656,7168.481656,7168.481656
2,Alecrim,3084.259098,3084.259098,3084.259098
3,Quintas,4174.864127,4174.864127,4174.864127
4,Cidade da Esperança,5617.458571,5617.458571,5617.458571
5,Lagoa Nova,2328.883186,2328.883186,2328.883186
6,Via Costeira,8940.126163,8940.126163,8940.126163
7,Ponta Negra,12192.168456,12192.168456,12192.168456
8,Capim Macio,6499.295650,6499.295650,6499.295650
9,Neópolis,6872.966013,6872.966013,6872.966013


In [7]:
arquivo = "emissions/emissions.csv"
df = pd.read_csv(arquivo)
df

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2025-05-14T15:57:32,Dijkstra Tradicional,d83594c6-c77b-41b2-afaf-762982f76e17,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,6.539132,7.144349e-05,0.000011,390.0,0.0,10.0,...,12,AMD Ryzen 5 5600G with Radeon Graphics,NaN,NaN,-35.2235,-5.8111,15.352566,machine,N,1.0
1,2025-05-14T15:57:34,Dijkstra MinHeap,59395787-0537-4060-90d6-4bb0a883d902,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,0.074773,8.051514e-07,0.000011,390.0,0.0,10.0,...,12,AMD Ryzen 5 5600G with Radeon Graphics,NaN,NaN,-35.2235,-5.8111,15.352566,machine,N,1.0
2,2025-05-14T15:57:36,OSMnx Dijkstra,94de18c9-256b-4443-aec4-56ebf4dde221,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,0.044472,4.717424e-07,0.000011,390.0,0.0,10.0,...,12,AMD Ryzen 5 5600G with Radeon Graphics,NaN,NaN,-35.2235,-5.8111,15.352566,machine,N,1.0
